In [1]:
import sys
import hail as hl
import pandas as pd

In [2]:
def remove_unnamed(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

## Make SAMPLE QC TABLE

DEFINE INPUT AND OUTPUT

In [23]:
SAMPLE_QC_FILE = "/mnt/i/UKB_DATA/imputed_UKB/qc_sphericity_index.tsv"
SAMPLE_QC_TABLE = "/mnt/i/UKB_DATA/imputed_UKB/qc_sphericity_index.kt"

READ SAMPLE QC

In [24]:
df = hl.import_table(SAMPLE_QC_FILE)

2022-05-16 01:10:38 Hail: INFO: Loading 64 fields. Counts by type:
  str: 64


In [25]:
df = df.annotate(PC1 = hl.float64(df['PC1']),
                 PC2 = hl.float64(df['PC2']),
                 PC3 = hl.float64(df['PC3']),
                 PC4 = hl.float64(df['PC4']),
                 PC5 = hl.float64(df['PC5']),
                 PC6 = hl.float64(df['PC6']),
                 PC7 = hl.float64(df['PC7']),
                 PC8 = hl.float64(df['PC8']),
                 PC9 = hl.float64(df['PC9']),
                 PC10 = hl.float64(df['PC10']),
                 age_at_MRI = hl.float64(df['age_at_MRI']),
                 eid = df['eid'],
                 in_white_British_ancestry_subset = df['ethnicity'] == "1.0",
                 used_in_pca_calculation = df['used.in.pca.calculation'] == "1.0",
                 no_excess_relatives = df['excess.relatives']=="0.0",
                 putative_sex_chromosome_aneuploidy = df['putative.sex.chromosome.aneuploidy']=="1.0",
                 isFemale = df['Inferred.Gender'] == '0.0')

In [26]:
df = df.filter(df.in_white_British_ancestry_subset==True)

In [27]:
df = df.filter(df.no_excess_relatives==True)

In [8]:
#df = df.filter(df.used_in_pca_calculation==True)

In [28]:
df=df.select('eid',
           'isFemale',
           'age_at_MRI',
           'PC1',
           'PC2',
           'PC3',
           'PC4',
           'PC5',
           'PC6',
           'PC7',
           'PC8',
           'PC9',
           'PC10')

In [29]:
df.write(SAMPLE_QC_TABLE,overwrite=True)

2022-05-16 01:10:58 Hail: INFO: wrote table with 22590 rows in 1 partition to /mnt/i/UKB_DATA/imputed_UKB/qc_sphericity_index.kt


In [30]:
df.count()

22590

## BUILD PIPLINES

In [31]:
import sys
import hail as hl
import pandas as pd

In [64]:
pheno = "BSA_length"
SAMPLE_QC_TABLE = '/mnt/i/UKB_DATA/imputed_UKB/qc_sphericity_index.kt'
kt = hl.read_table(SAMPLE_QC_TABLE)
n_samples = kt.count()

PHESANT_FILE = f"/mnt/i/UKB_DATA/tsv_pheno/{pheno}.tsv"

traits = hl.import_table(PHESANT_FILE,impute=True)
traits = traits.annotate(idx = hl.str(traits.idx), pheno=hl.float64(traits.pheno))


traits=traits.key_by('idx')
kt=kt.key_by('eid')
kt=kt.annotate(pheno = traits[kt.eid].pheno)
kt.write(f"/mnt/i/UKB_DATA/pipelines/{pheno}.kt",overwrite=True)

22590


2022-05-16 01:29:29 Hail: INFO: Reading table to impute column types
2022-05-16 01:29:30 Hail: INFO: Finished type imputation
  Loading field '' as type int32 (imputed)
  Loading field 'Unnamed: 0' as type int32 (imputed)
  Loading field 'idx' as type int32 (imputed)
  Loading field 'pheno' as type float64 (imputed)
2022-05-16 01:29:30 Hail: INFO: Coerced sorted dataset
2022-05-16 01:29:30 Hail: INFO: Coerced sorted dataset
2022-05-16 01:29:31 Hail: INFO: wrote table with 22590 rows in 1 partition to /mnt/i/UKB_DATA/pipelines/BSA_length.kt
2022-05-16 01:43:26 Hail: INFO: Reading table to impute column types
2022-05-16 01:43:28 Hail: INFO: Finished type imputation
  Loading field '' as type int32 (imputed)
  Loading field 'Unnamed: 0' as type int32 (imputed)
  Loading field 'idx' as type int32 (imputed)
  Loading field 'pheno' as type float64 (imputed)
2022-05-16 01:43:29 Hail: INFO: Coerced sorted dataset
2022-05-16 01:43:29 Hail: INFO: Coerced sorted dataset
2022-05-16 01:43:30 Hail: 

In [63]:
kt.show(10)

2022-05-16 01:29:18 Hail: INFO: Coerced sorted dataset
2022-05-16 01:29:19 Hail: INFO: Coerced sorted dataset


,,,,,,,,,,,,,
eid,isFemale,age_at_MRI,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,pheno
str,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
"""1000158""",False,4.40e+01,-1.16e+01,2.74e+00,-1.26e+00,1.58e+00,-7.08e+00,-3.52e+00,9.70e-01,-3.36e+00,1.05e+00,1.33e+00,4.33e+01
"""1000243""",False,5.90e+01,-1.39e+01,4.20e+00,-3.87e+00,3.94e-01,1.53e+00,-1.22e+00,-4.83e-01,-2.18e+00,1.72e+00,-7.86e-01,5.09e+01
"""1000262""",True,5.80e+01,-1.46e+01,5.00e+00,-1.88e+00,4.13e+00,-5.70e-01,5.44e-01,-1.30e+00,1.67e+00,-4.42e+00,1.21e+00,4.44e+01
"""1000489""",False,6.60e+01,-1.13e+01,4.54e+00,-3.70e+00,2.77e+00,-8.50e+00,-1.19e+00,-1.90e-01,-1.82e+00,8.88e+00,5.65e+00,5.23e+01
"""1000563""",True,4.90e+01,-1.36e+01,3.44e+00,-1.51e+00,2.02e+00,5.62e+00,-3.65e+00,2.92e+00,-8.56e-01,-2.60e+01,4.15e+00,4.48e+01
"""1000670""",False,6.80e+01,-1.22e+01,4.78e+00,-1.68e+00,-9.45e-02,-5.13e+00,1.35e+00,-1.17e+00,-2.67e+00,-3.86e+00,8.03e-01,4.62e+01
"""1001023""",False,5.80e+01,-1.26e+01,4.62e+00,-9.56e-01,9.22e-01,1.01e+00,3.02e+00,-5.58e-01,2.69e-01,-8.24e+00,7.87e-01,4.95e+01
"""1001525""",False,6.20e+01,-1.28e+01,5.11e+00,-1.49e+00,4.08e+00,9.27e+00,2.14e+00,-8.49e-02,2.08e-01,2.99e+00,-6.30e-01,4.98e+01


In [59]:
traits.show(10)

,,,
,Unnamed: 0,idx,pheno
int32,int32,str,float64
0,0,"""1000158""",4.33e+01
1,1,"""1000243""",5.09e+01
2,2,"""1000262""",4.44e+01
3,3,"""1000369""",5.23e+01
4,4,"""1000489""",5.23e+01
5,5,"""1000490""",5.21e+01
6,6,"""1000542""",3.76e+01
7,7,"""1000563""",4.48e+01


In [46]:
kt.count()

22590

In [43]:
table_joined.show(10)

2022-05-16 01:19:52 Hail: INFO: Coerced sorted dataset
2022-05-16 01:19:52 Hail: INFO: Coerced sorted dataset


,,,,,,,,,,,,,,,
idx,,Unnamed: 0,pheno,isFemale,age_at_MRI,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
str,int32,int32,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
"""1000158""",0,0,4.33e+01,False,4.40e+01,-1.16e+01,2.74e+00,-1.26e+00,1.58e+00,-7.08e+00,-3.52e+00,9.70e-01,-3.36e+00,1.05e+00,1.33e+00
"""1000243""",1,1,5.09e+01,False,5.90e+01,-1.39e+01,4.20e+00,-3.87e+00,3.94e-01,1.53e+00,-1.22e+00,-4.83e-01,-2.18e+00,1.72e+00,-7.86e-01
"""1000262""",2,2,4.44e+01,True,5.80e+01,-1.46e+01,5.00e+00,-1.88e+00,4.13e+00,-5.70e-01,5.44e-01,-1.30e+00,1.67e+00,-4.42e+00,1.21e+00
"""1000489""",4,4,5.23e+01,False,6.60e+01,-1.13e+01,4.54e+00,-3.70e+00,2.77e+00,-8.50e+00,-1.19e+00,-1.90e-01,-1.82e+00,8.88e+00,5.65e+00
"""1000563""",7,7,4.48e+01,True,4.90e+01,-1.36e+01,3.44e+00,-1.51e+00,2.02e+00,5.62e+00,-3.65e+00,2.92e+00,-8.56e-01,-2.60e+01,4.15e+00
"""1000670""",8,8,4.62e+01,False,6.80e+01,-1.22e+01,4.78e+00,-1.68e+00,-9.45e-02,-5.13e+00,1.35e+00,-1.17e+00,-2.67e+00,-3.86e+00,8.03e-01
"""1001023""",10,10,4.95e+01,False,5.80e+01,-1.26e+01,4.62e+00,-9.56e-01,9.22e-01,1.01e+00,3.02e+00,-5.58e-01,2.69e-01,-8.24e+00,7.87e-01
"""1001525""",15,15,4.98e+01,False,6.20e+01,-1.28e+01,5.11e+00,-1.49e+00,4.08e+00,9.27e+00,2.14e+00,-8.49e-02,2.08e-01,2.99e+00,-6.30e-01
